In [1]:
import aiohttp
import asyncio
import bs4
import datetime
from io import BytesIO
import pandas as pd
import PyPDF2
import requests
import time
from tqdm.contrib.telegram import tqdm
from transliterate import to_latin
import json

In [2]:
TOKEN = os.getenv('TOKEN')
CHAT_ID = os.getenv('CHAT_ID')

In [3]:
NEWS_URL = "https://www.natlib.uz/bbs/list/1?pn=5&oi=insert_date&os=desc" #1250
NEWS_LAST = 425

In [4]:
URLS = [[NEWS_URL, NEWS_LAST], [MEETING_URL, MEETING_LAST], [FLY_URL, FLY_LAST], 
        [DELEG_URL, DELEG_LAST], [CONGR_URL, CONGR_LAST], [RULES_URL, RULES_LAST], [RULES2_URL, RULES2_LAST]]

In [6]:
BASE_URL = "https://www.natlib.uz/"

In [7]:
from bs4 import BeautifulSoup

response = requests.get(NEWS_URL)
soup = BeautifulSoup(response.text, 'lxml')
content = soup.find('ul', class_="thumbList").find_all('li')

# print(content)

for i in content:
    print(BASE_URL + i.find('a')['href'])

https://www.natlib.uz//bbs/content/1_557434?os=desc&pn=5&oi=insert_date&
https://www.natlib.uz//bbs/content/1_557433?os=desc&pn=5&oi=insert_date&
https://www.natlib.uz//bbs/content/1_557429?os=desc&pn=5&oi=insert_date&
https://www.natlib.uz//bbs/content/1_557428?os=desc&pn=5&oi=insert_date&
https://www.natlib.uz//bbs/content/1_557432?os=desc&pn=5&oi=insert_date&
https://www.natlib.uz//bbs/content/1_557431?os=desc&pn=5&oi=insert_date&


In [8]:
links = []

In [10]:
async def main():
    async with aiohttp.ClientSession() as session:
        for page in tqdm(range(1, NEWS_LAST+1), token=TOKEN, chat_id=CHAT_ID):
            try:
                async with session.get(NEWS_URL.format(page=page)) as resp:
                    if resp.status == 200:
                        data = await resp.text()
                        soup = bs4.BeautifulSoup(data)
                        selected_links = soup.find('ul', class_="thumbList").find_all('li')
                        links.extend([BASE_URL+link.find('a')['href'] for link in selected_links])
                    elif resp.status != 404:
                        await asyncio.sleep(60) # 1 minute
            except Exception as e:
                print(e, "page_num", page)
await main()

  0%|          | 0/425 [00:00<?, ?it/s]

In [12]:
results = []

async def main():
    async with aiohttp.ClientSession() as session:

        for ind, link in enumerate(tqdm(links, token=TOKEN, chat_id=CHAT_ID)):
            try:
                async with session.get(link) as resp:
                    if resp.status == 200:
                        data = await resp.text()
                        soup = bs4.BeautifulSoup(data)

                        title = soup.find('p', class_="boardInfoTitle").text
                        content = soup.find('div', class_="boardContent").text

                        results.append({
                            "title": title.strip(),
                            "content": content.strip()
                        })
                    elif resp.status != 404:
                        await asyncio.sleep(60) # 1 minute
                        print("404", link)
            except Exception as e:
                print(e, link)
    
await main()

  0%|          | 0/2550 [00:00<?, ?it/s]

In [13]:
df = pd.DataFrame(results)

In [18]:
df = df.replace({'\n':''}, regex=True)

In [14]:
from transliterate import to_latin

df['title'] = df['title'].apply(lambda x: to_latin(x))

df['content'] = df['content'].apply(lambda x: to_latin(x))

In [16]:
import demoji

df['title'] = df['title'].apply(lambda x: demoji.replace(x, ""))

df['content'] = df['content'].apply(lambda x: demoji.replace(x, ""))

In [19]:
df

,title,content
0,“Yilning eng yaxshi Axborot-kutubxona mutaxass...,Axborot-kutubxona mutaxassislarining kasbiy ma...
1,"11-iyul - ""Xalqaro shokolad kuni""","Bugun, 11-iyul kuni dunyo bo‘ylab Xalqaro shok..."
2,QIZIQARLI MA’LUMOTLAR,#Bilimlarga_boy_dushanba Siz qaysi turdagi kit...
3,Bilimlarga boy dushanba,#Bilimlarga_boy_dushanba qiziqarli maʼlumotlar...
4,Alisher Navoiy nomidagi O‘zbekiston Milliy kut...,2023-yilning 10 iyul kuni Alisher Navoiy nomid...
...,...,...
2545,"11-iyul - ""Xalqaro shokolad kuni""","Bugun, 11-iyul kuni dunyo bo‘ylab Xalqaro shok..."
2546,QIZIQARLI MA’LUMOTLAR,#Bilimlarga_boy_dushanba Siz qaysi turdagi kit...
2547,Bilimlarga boy dushanba,#Bilimlarga_boy_dushanba qiziqarli maʼlumotlar...
2548,Alisher Navoiy nomidagi O‘zbekiston Milliy kut...,2023-yilning 10 iyul kuni Alisher Navoiy nomid...


In [27]:
pd.DataFrame(results)

,title,content
0,Namangan viloyati IIB boshligʻi huzurida Chust...,"Unda Boshqarma boshligʻi oʻrinbosarlari, Boshq..."
1,Namangan viloyati IIB Majlislar zalida Migrats...,"Unda, viloyat IIB boshligʻi oʻrinbosarlari, so..."
2,Denov tumanida tashkilotlar oʻrtasida sportnin...,Joriy yilning 15-iyul kuni Denov tumanidagi ma...
3,Surxondaryo viloyati IIB Migratsiya va fuqarol...,Joriy yilning 15-iyul kuni Surxondaryo viloyat...
4,Qumqoʻrgʻon tumani IIBda yangi “Kutubxona” foy...,2023-yil 14-iyul kuni Qumqoʻrgʻon tumani Ichki...
...,...,...
1263,“14 yanvar` – Vatan himoyachilari kuni” bayram...,Barcha joyda bo'lgani kabi O'zbekiston Respubl...
1264,Guliston shahrida “Profilaktika kuni” munosaba...,Guliston shahar Qurilish kasb-hunar kollejida ...
1265,Bir guruh ichki ishlar organlari xodimlari taq...,Ichki ishlar vazirligi tizimi xodimlarining ke...
1266,"""Vatanparvarlik oyligi"" davom etmoqda",IIV Akademiyasi Axborot xizmatining xabar beri...


In [20]:
df.to_pickle("MainFiles/natib.pickle")